# quant-econ Solutions: The Lucas Asset Pricing Model

Solutions for http://quant-econ.net/py/markov_asset.html

In [8]:
import numpy as np
import quantecon as qe

Next we load the code from the lectures into a cell to have access to the functions

In [9]:
# %load https://raw.githubusercontent.com/QuantEcon/QuantEcon.applications/master/markov_asset/asset_pricing.py
"""
Filename: asset_pricing.py

Computes asset prices in a Lucas endowment economy when the endowment obeys
geometric growth driven by a finite state Markov chain.  That is,

.. math::
    d_{t+1} = X_{t+1} d_t

where :math:`\{X_t\}` is a finite Markov chain with transition matrix P.

References
----------

    http://quant-econ.net/py/markov_asset.html

"""
import numpy as np
from numpy.linalg import solve


class AssetPriceModel:
    r"""
    A class that stores the primitives of the asset pricing model, plus a few
    useful matrices as attributes

    Parameters
    ----------
    beta : scalar, float
        Discount factor
    mc : MarkovChain
        Contains the transition matrix and set of state values for the state
        proces
    gamma : scalar(float)
        Coefficient of risk aversion

    Attributes
    ----------
    beta, mc, gamm : as above

    P_tilde : ndarray
        The matrix :math:`P(x, y) y^(1 - \gamma)`

    P_tilde : ndarray
        The matrix :math:`P(x, y) y^(\gamma)`

    """
    def __init__(self, beta, mc, gamma):
        self.beta, self.mc, self.gamma = beta, mc, gamma
        self.n = self.mc.P.shape[0]

    @property
    def P_tilde(self):
        P = self.mc.P
        y = self.mc.state_values
        return P * y**(1 - self.gamma)  # using broadcasting

    @property
    def P_check(self):
        P = self.mc.P
        y = self.mc.state_values
        return P * y**(-self.gamma)  # using broadcasting



def tree_price(apm):
    """
    Computes the price-dividend ratio of the Lucas tree.

    Parameters
    ----------
    apm: AssetPriceModel
        An instance of AssetPriceModel containing primitives

    Returns
    -------
    v : array_like(float)
        Lucas tree price-dividend ratio

    """
    # == Simplify names == #
    beta = apm.beta
    P_tilde = apm.P_tilde

    # == Compute v == #
    I = np.identity(apm.n)
    O = np.ones(apm.n)
    v = beta * solve(I - beta * P_tilde, P_tilde @ O)

    return v


def consol_price(apm, zeta):
    """
    Computes price of a consol bond with payoff zeta

    Parameters
    ----------
    apm: AssetPriceModel
        An instance of AssetPriceModel containing primitives

    zeta : scalar(float)
        Coupon of the console

    Returns
    -------
    p_bar : array_like(float)
        Console bond prices

    """
    # == Simplify names == #
    beta = apm.beta

    # == Compute price == #
    P_check = apm.P_check
    I = np.identity(apm.n)
    O = np.ones(apm.n)
    p_bar = beta * solve(I - beta * P_check, P_check.dot(zeta * O))

    return p_bar


def call_option(apm, zeta, p_s, T=[], epsilon=1e-8):
    """
    Computes price of a call option on a consol bond, both finite
    and infinite horizon

    Parameters
    ----------
    apm: AssetPriceModel
        An instance of AssetPriceModel containing primitives

    zeta : scalar(float)
        Coupon of the console

    p_s : scalar(float)
        Strike price

    T : iterable(integers)
        Length of option in the finite horizon case

    epsilon : scalar(float), optional(default=1e-8)
        Tolerance for infinite horizon problem

    Returns
    -------
    w_bar : array_like(float)
        Infinite horizon call option prices

    w_bars : dict
        A dictionary of key-value pairs {t: vec}, where t is one of
        the dates in the list T and vec is the option prices at that
        date

    """
    # == Simplify names, initialize variables == #
    beta = apm.beta
    P_check = apm.P_check

    # == Compute consol price == #
    v_bar = consol_price(apm, zeta)

    # == Compute option price == #
    w_bar = np.zeros(apm.n)
    error = epsilon + 1
    t = 0
    w_bars = {}
    while error > epsilon:
        if t in T:
            w_bars[t] = w_bar

        # == Maximize across columns == #
        to_stack = (beta*P_check.dot(w_bar), v_bar-p_s)
        w_bar_new = np.amax(np.vstack(to_stack), axis=0)

        # == Find maximal difference of each component == #
        error = np.amax(np.abs(w_bar-w_bar_new))

        # == Update == #
        w_bar = w_bar_new
        t += 1

    return w_bar, w_bars


## Exercise 1

First let's enter the parameters:

In [11]:
# parameters for object
n = 5
P = 0.0125 * np.ones((n, n))
P += np.diag(0.95 - 0.0125 * np.ones(5))
s = np.array([1.05, 1.025, 1.0, 0.975, 0.95])  # state values
mc = qe.MarkovChain(P, state_values=s)

gamma = 2.0
beta = 0.94
zeta = 1.0
p_s = 150.0

Next we'll create an instance of `AssetPriceModel` to feed into the functions.

In [12]:
apm = AssetPriceModel(beta, mc, gamma)

Now we just need to call the relevent functions on the data:

In [13]:
tree_price(apm)

array([ 12.72221763,  14.72515002,  17.57142236,  21.93570661,  29.47401578])

In [14]:
consol_price(apm, zeta)

array([  87.56860139,  109.25108965,  148.67554548,  242.55144082,
        753.87100476])

In [15]:
call_option(apm, zeta, p_s)

(array([  64.30843769,   80.05179282,  108.67734545,  176.83933585,
         603.87100476]), {})